In [ ]:
using DelimitedFiles;
using JSON
using DelimitedFiles;
using Plots, LaTeXStrings;
using Printf

### PLOTTING ###

function read_run_results(run_number)
    U_list = []
    M_list = []
    run="new_MFG_convergence_rate_runs/run"*string(run_number)
    params = JSON.parsefile(run*"_params.json")
    Δt = params["Δt"]
    for h in params["h_list"]
        push!(U_list, readdlm(run*"_U_mat_conv_h$(h)_deltat$(Δt).csv", ','))
        push!(M_list, readdlm(run*"_M_mat_conv_h$(h)_deltat$(Δt).csv", ','))
    end
    return params, U_list, M_list
end

function read_run_iterations(run_number, h)
    U_iter_list = []
    M_iter_list = []
    run="new_MFG_convergence_rate_runs/run"*string(run_number)
    params = JSON.parsefile(run*"_params.json")
    Δt = params["Δt"]

    files = readdir("new_MFG_iteration_plots")
    
    iter_numbers = Int[]

    for f in files
        if occursin("run$(run_number)_iter", f) && occursin("M_mat_conv_h$(h)", f)
            m =  match(r"_iter(\d+)", f)
            if m !== nothing
                push!(iter_numbers, parse(Int, m.captures[1]))
            end
        end
    end
    println("iter_numbers: ", iter_numbers)
    iter_folder="new_MFG_iteration_plots/run"*string(run_number)
    final_iter = maximum(iter_numbers)
    for iter in 0:final_iter
        push!(U_iter_list, readdlm(iter_folder*"_iter$(iter)_U_mat_conv_h$(h)_deltat$(Δt).csv", ','))
        push!(M_iter_list, readdlm(iter_folder*"_iter$(iter)_M_mat_conv_h$(h)_deltat$(Δt).csv", ','))
    end
    return params, U_iter_list, M_iter_list, final_iter
end

params, U_list, M_list = read_run_results(1)
println("params: ", params)
println("size(U_list[1]): ", size(U_list[1]))

#PLOT RESULTS


function plot_iterations(run_number, h, save_figures=false)
    params, U_list, M_list, final_iter = read_run_iterations(run_number, h)
    (h_list, α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R) = (params["h_list"], params["α"], params["x_l"], params["x_r"], params["Δt"], params["t_0"], params["T"], params["ν"], params["num_it_MFG"], params["num_it_HJB"],  params["δ"], params["R"])
    
    println("Plot results for run number: ", run_number) 
    println("Plot iterations from h=",h)
    println("!!")
    println("parameters: ", params)
    println("size(U_list): ", size(U_list))
    flush(stdout)

    final_iter = #largest iteration for this run number and this h.
    for i in 0:final_iter
        #println("----")
        #println("i: ", i)
        x_vec = x_l:h:(x_r-h);
        N_h =length(x_vec);
        t_vec = t_0:Δt:(T-Δt)
        N_T =length(t_vec)
        #println("x_vec: ", x_vec)
        #println("N_h: ", N_h)
        #println("t_vec: ", t_vec)
        #println("N_T: ", N_T)

        plot_U_mat = transpose(reverse(transpose(U_list[i+1]), dims=2));
        plot_M_mat = transpose(reverse(transpose(M_list[i+1]), dims=2));

        #println("i: ", i, "SUM: ", sum(plot_M_mat[:,end]*h))
        
        zticks1 = 5:5:20
        p1 = plot(t_vec, x_vec, plot_M_mat, ylabel=L"x", xlabel=L"t",  st=:surface, labelfontsize=15, zlim=(0,50),
        color=cgrad(:cool, rev=false), size=(500, 500), xlim=(0,t_vec[end]), ylim=(0.1,0.9), legend=false, camera=(90- 20, 20), xflip=true, zticks=zticks1, title=L"m(x,t)")
        zticks2 = 0.5:0.5:4
        p2 = plot(t_vec, x_vec, plot_U_mat, xlabel=L"t", ylabel=L"x",   st=:surface, labelfontsize=15,
        color=:cividis, size=(500, 500), xlim=(0,t_vec[end]), zlim=(0, 4), legend=false, camera=(90- 20, 40), xflip=true, zticks=zticks2, title=L"u(x,t)",  ylim=(0, 1))

        # Save plots as PDF with h in the filename
        if save_figures
            savefig(p1, joinpath("figures",run*"_plot_new_M_h$(h)_deltat$(Δt).pdf"))
            savefig(p2, joinpath("figures",run*"_plot_new_U_h$(h)_deltat$(Δt).pdf"))
        end 
        p = plot(p1, p2, layout=2, size=(600, 300), title="iter"*string(i)*"_h="*string(h), titlefontsize=10)
        #println("Display:") 
        display(p)
        # println("i: ", i)
        # if i==1
        #     break
        # end
        #println("!!!!!!!!!")
    end
end


function plot_results(run_number, h_start=1, h_end=-1, save_figures=false)
    params, U_list, M_list = read_run_results(run_number)
    (h_list, α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R) = (params["h_list"], params["α"], params["x_l"], params["x_r"], params["Δt"], params["t_0"], params["T"], params["ν"], params["num_it_MFG"], params["num_it_HJB"],  params["δ"], params["R"])
    
    if h_end==-1
        h_end=length(h_list)
    end
    println("Plot results for run number: ", run_number) 
    println("!!")
    println("parameters: ", params)
    println("size(U_list): ", size(U_list))
    flush(stdout)
    println("h_list[h_start:h_end]: ", h_list[h_start:h_end])
    for (i,h) in enumerate(h_list[1:h_end])
        if i<h_start 
            continue
        end
        println("----")
        println("i: ", i)
        x_vec = x_l:h:(x_r-h);
        N_h =length(x_vec);
        t_vec = t_0:Δt:(T-Δt)
        N_T =length(t_vec)
        println("x_vec: ", x_vec)
        println("N_h: ", N_h)
        println("t_vec: ", t_vec)
        println("N_T: ", N_T)

        x_reverse_plot = false
        if x_reverse_plot        
            plot_U_mat = transpose(reverse(transpose(U_list[i]), dims=2));
            plot_M_mat = transpose(reverse(transpose(M_list[i]), dims=2));
        else 
            plot_U_mat = (U_list[i]);
            plot_M_mat = (M_list[i]);
        end

        println("SUM: ", sum(plot_M_mat[:,end]*h))
        

        a1=20
        a4=25
        a2=30
        a3=40

        p1 = plot(t_vec, x_vec, reverse(plot_M_mat, dims=1), ylabel=L"x", xlabel=L"t",  st=:surface, guidefont = font(20), tickfont = font(12), legendfont = font(14), titlefont=font(20),
        color=cgrad(:cool, rev=false), size=(750, 750), xlim=(0,t_vec[end]), ylim=(0,1), legend=false, camera=(90- a1, a2), xflip=true, title=L"m(x,t)", zlim=(0,20))
        p2 = plot(t_vec, x_vec, reverse(plot_U_mat, dims=1), xlabel=L"t", ylabel=L"x",   st=:surface, guidefont = font(20), tickfont = font(12), legendfont = font(14), titlefont=font(20),
        color=:cividis, size=(750, 750), xlim=(0,t_vec[end]), legend=false, camera=(90- a4, a3), xflip=true,  title=L"u(x,t)",  ylim=(0, 1), zlim=(0,4))

        # Save plots as PDF with h in the filename
        if save_figures
            run="run"*string(run_number)
            if x_reverse_plot
                savefig(p1, joinpath("figures",run*"_plot_new_M_h$(h)_deltat$(Δt).pdf"))
                savefig(p2, joinpath("figures",run*"_plot_new_U_h$(h)_deltat$(Δt).pdf"))
            else 
                savefig(p1, joinpath("figures",run*"_plot_new_M_h$(h)_deltat$(Δt)_not_rev.pdf"))
                savefig(p2, joinpath("figures",run*"_plot_new_U_h$(h)_deltat$(Δt)_not_rev.pdf"))
            end
        end 
        p = plot(p1, p2, layout=2, size=(600, 300), title="h="*string(h), titlefontsize=10)
        println("Display:") 
        display(p1)
        display(p2)
        # println("i: ", i)
        # if i==1
        #     break
        # end
        println("!!!!!!!!!")
        

    end
end


params: Dict{String, Any}("T" => 1, "δ" => 0.4, "x_l" => -1, "h_list" => Any[0.015625, 0.0078125], "x_r" => 2, "ν" => 0.0081, "α" => 1.5, "num_it_HJB" => 20, "Δt" => 0.01, "num_it_MFG" => 50, "R" => 30, "t_0" => 0)
size(U_list[1]): (192, 100)


plot_results (generic function with 4 methods)

In [ ]:
using DelimitedFiles;
using JSON
using DelimitedFiles;
using LaTeXStrings;
using Printf
using CairoMakie
using Colors


### PLOTTING WITH COLOR ###

function read_run_results(run_number)
    U_list = []
    M_list = []
    run="new_MFG_convergence_rate_runs/run"*string(run_number)
    params = JSON.parsefile(run*"_params.json")
    Δt = params["Δt"]
    for h in params["h_list"]
        push!(U_list, readdlm(run*"_U_mat_conv_h$(h)_deltat$(Δt).csv", ','))
        push!(M_list, readdlm(run*"_M_mat_conv_h$(h)_deltat$(Δt).csv", ','))
    end
    return params, U_list, M_list
end

function plot_results_makie(run_number, h_start=1, h_end=-1, save_figures=false)
    params, U_list, M_list = read_run_results(run_number)
    (h_list, α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R) = (params["h_list"], params["α"], params["x_l"], params["x_r"], params["Δt"], params["t_0"], params["T"], params["ν"], params["num_it_MFG"], params["num_it_HJB"],  params["δ"], params["R"])
    
    if h_end==-1
        h_end=length(h_list)
    end
    println("Plot results for run number: ", run_number) 
    println("!!")
    println("parameters: ", params)
    println("size(U_list): ", size(U_list))
    flush(stdout)
    for (i,h) in enumerate(h_list[h_start:h_end])
        if i<h_start 
            continue
        end
        x_vec = x_l:h:(x_r-h);
        N_h =length(x_vec);
        t_vec = t_0:Δt:(T-Δt)
        N_T =length(t_vec)
        plot_U_mat = transpose(reverse(transpose(U_list[i]), dims=2));
        plot_M_mat = transpose(reverse(transpose(M_list[i]), dims=2));

        x_vec_trimmed = x_vec[0.0 .<= x_vec .< 1]
        println("size(x_vec_trimmed): ",size(x_vec_trimmed))
     
        plot_M_mat_trimmed = plot_M_mat[0.0 .<= x_vec .< 1,:]
        plot_U_mat_trimmed = plot_U_mat[0.0 .<= x_vec .< 1,:]


        cm = cgrad([:darkblue, :aqua, :yellow, :orange], [0, 0.1, 0.6, 0.9], scale=:linear)

        #M:
        fig = Figure()
        ax = Axis3(fig[1, 1], azimuth= (-0.5-0.1)* pi,  elevation= 0.14*pi)
        #ax = Axis3(fig[1, 1], azimuth= (1)* pi,  elevation= 0.0) # -> There is actually some undulation in the z direction
        surf = surface!(ax, x_vec_trimmed, t_vec, plot_M_mat_trimmed, colormap = cm)
        xlims!(ax,0,1)
        ax.xticks = 0:0.25:1
        ylims!(ax,0,2)
        zlims!(ax,0,20)
        ax.xlabel = L"x"
        ax.ylabel = L"t"
        ax.zlabel=""
        # Control aspect ratio (relative scaling of x,y,z)
        ax.aspect = (1, 1, 0.5)
        ax.title=L"m(x,t)"

        ax.xlabelsize = 26   # font size for x label
        ax.ylabelsize = 26
        ax.zlabelsize = 26

        # Title font size
        ax.titlesize = 40

        # Tick labels
        ax.xticklabelsize = 22
        ax.yticklabelsize = 22
        ax.zticklabelsize = 22
        Colorbar(fig[1, 2], surf, ticklabelsize=22)
        

        #U: 
        cm2 = cgrad([:darkblue, :aqua, :yellow, :orange])

        fig2 = Figure()
        ax2 = Axis3(fig2[1, 1], azimuth= (-0.5-0.15)* pi,  elevation= 0.17*pi)
        surf2 = surface!(ax2, x_vec_trimmed, t_vec, plot_U_mat_trimmed, colormap = cm2, colorrange=(0,2.5))
        xlims!(ax2,0,1)
        ax2.xticks = 0:0.25:1
        ylims!(ax2,0,2)
        zlims!(ax2,0,3)
        ax2.zticks = 0:1:3
        ax2.xlabel = L"x"
        ax2.ylabel = L"t"
        ax2.zlabel=""
        # Control aspect ratio (relative scaling of x,y,z)
        ax2.aspect = (1, 1, 0.5)
        ax2.title=L"u(x,t)"

        ax2.xlabelsize = 26   # font size for x label
        ax2.ylabelsize = 26
        ax2.zlabelsize = 26

        # Title font size
        ax2.titlesize = 40

        # Tick labels
        ax2.xticklabelsize = 21
        ax2.yticklabelsize = 22
        ax2.zticklabelsize = 22
        Colorbar(fig2[1, 2], surf2, ticklabelsize=22)

        
        if save_figures
            save(joinpath("figures","run"*string(run_number)*"_plot2_olav_ex_M_h$(h)_deltat$(Δt).png"), fig)
            save(joinpath("figures","run"*string(run_number)*"_plot2_olav_ex_U_h$(h)_deltat$(Δt).png"), fig2)
        end

        display(fig)
        display(fig2)
        end
end

plot_results_makie (generic function with 4 methods)

In [34]:

#PLOT RESULTS AT FINAL TIME

function plot_final_time_results(run_number, h_start=1, h_end=-1,  save_figures=false)
    params, U_list, M_list = read_run_results(run_number)
    (h_list, α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R) = (params["h_list"], params["α"], params["x_l"], params["x_r"], params["Δt"], params["t_0"], params["T"], params["ν"], params["num_it_MFG"], params["num_it_HJB"],  params["δ"], params["R"])
    
    if h_end==-1
        h_end = length(h_list)
    end
    println("FINAL TIME PLOTS")
    println("Plot results for run number: ", run_number) 
    println("!!")
    println("parameters: ", params)
    println("size(U_list): ", size(U_list))
    flush(stdout)
    for (i,h) in enumerate(h_list[h_start: h_end])
        println("----")
        println("i: ", i)
        x_vec = x_l:h:(x_r-h);
        N_h =length(x_vec);
        t_vec = t_0:Δt:(T-Δt)
        N_T =length(t_vec)
        println("x_vec: ", x_vec)
        println("N_h: ", N_h)
        println("t_vec: ", t_vec)
        println("N_T: ", N_T)

        ###################################
        time_index = div(length(t_vec),1)
        println("time_index: ", time_index)

        plot_U_mat_final = U_list[i][:,1];
        plot_M_mat_final = M_list[i][:,time_index];
        plot_U_mat_first = U_list[i][:,time_index];
        plot_M_mat_first = M_list[i][:,1];

        p1 = plot(x_vec, plot_M_mat_final, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="M, h="*string(h)*", t="*string(t_vec[time_index]),
        color=cgrad(:cool, rev=false), size=(500, 500), legend=false) #, xlim=(0,1) )
        #zticks2 = 0.5:0.5:4
        p2 = plot(x_vec, plot_U_mat_final, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="U, h="*string(h)*", t="*string(t_vec[1]),
        color=:cividis, size=(500, 500), legend=false) #,xlim=(0, 1),)

        p3 = plot(x_vec, plot_M_mat_first, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="M, h="*string(h)*", t="*string(t_vec[1]),
        color=cgrad(:cool, rev=false), size=(500, 500), legend=false) #, xlim=(0,1) )
        #zticks2 = 0.5:0.5:4
        
        p4 = plot(x_vec, plot_U_mat_first, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="U, h="*string(h)*", t="*string(t_vec[time_index]),
        color=:cividis, size=(500, 500), legend=false) #,xlim=(0, 1),)

        if save_figures
            savefig(p1, joinpath("figures",run*"_final_time_plot_final_M_h$(h)_deltat$(Δt).pdf"))
            savefig(p2, joinpath("figures",run*"_final_time_plot_final_U_h$(h)_deltat$(Δt).pdf"))
            savefig(p3, joinpath("figures",run*"_final_time_plot_first_M_h$(h)_deltat$(Δt).pdf"))
            savefig(p4, joinpath("figures",run*"_final_time_plot_first_U_h$(h)_deltat$(Δt).pdf"))
        end
        p = plot(p1, p2, p3, p4, layout=4, size=(600, 600), titlefontsize=10)
        println("Display:")
        display(p)
        # println("i: ", i)
        # if i==1
        #     break
        # end
        println("!!!!!!!!!")
    end
end

plot_final_time_results (generic function with 4 methods)

In [78]:

#PLOT RESULTS AT FINAL TIME

function plot_final_time_iterations(run_number, h)
       params, U_list, M_list, final_iter = read_run_iterations(run_number, h)
    (h_list, α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R) = (params["h_list"], params["α"], params["x_l"], params["x_r"], params["Δt"], params["t_0"], params["T"], params["ν"], params["num_it_MFG"], params["num_it_HJB"],  params["δ"], params["R"])
    
    println("Plot results for run number: ", run_number) 
    println("Plot iterations from h=",h)
    println("!!")
    println("parameters: ", params)
    println("size(U_list): ", size(U_list))
    flush(stdout)

    final_iter = #largest iteration for this run number and this h.
    for i in 0:final_iter
        #println("----")
        #println("i: ", i)
        x_vec = x_l:h:(x_r-h);
        N_h =length(x_vec);
        t_vec = t_0:Δt:(T-Δt)
        N_T =length(t_vec)
        #println("x_vec: ", x_vec)
        #println("N_h: ", N_h)
        #println("t_vec: ", t_vec)
        #println("N_T: ", N_T)

        plot_U_mat = transpose(reverse(transpose(U_list[i+1]), dims=2));
        plot_M_mat = transpose(reverse(transpose(M_list[i+1]), dims=2));

        ###################################
        time_index = div(length(t_vec),1)
        println("time_index: ", time_index)

        plot_U_mat_final = U_list[i+1][:,1];
        plot_M_mat_final = M_list[i+1][:,time_index];
        plot_U_mat_first = U_list[i+1][:,time_index];
        plot_M_mat_first = M_list[i+1][:,1];

        p1 = plot(x_vec, plot_M_mat_final, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="M, h="*string(h)*", t="*string(t_vec[time_index]),
        color=cgrad(:cool, rev=false), size=(500, 500), legend=false) #, xlim=(0,1) )
        #zticks2 = 0.5:0.5:4
        p2 = plot(x_vec, plot_U_mat_final, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="U, h="*string(h)*", t="*string(t_vec[1]),
        color=:cividis, size=(500, 500), legend=false) #,xlim=(0, 1),)

        p3 = plot(x_vec, plot_M_mat_first, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="M, h="*string(h)*", t="*string(t_vec[1]),
        color=cgrad(:cool, rev=false), size=(500, 500), legend=false) #, xlim=(0,1) )
        #zticks2 = 0.5:0.5:4
        
        p4 = plot(x_vec, plot_U_mat_first, xlabel=L"x", ylabel=L"y", labelfontsize=10,title="U, h="*string(h)*", t="*string(t_vec[time_index]),
        color=:cividis, size=(500, 500), legend=false) #,xlim=(0, 1),)

        p = plot(p1, p2, p3, p4, layout=4, size=(600, 600), title="iter"*string(i)*"_h="*string(h), titlefontsize=10)
        println("Display:")
        display(p)
        # println("i: ", i)
        # if i==1
        #     break
        # end
        println("!!!!!!!!!")
    end
end

plot_final_time_iterations (generic function with 1 method)

In [36]:
# Calculate convergence rate


#CONVERGENCE RATE CALCULATION FUNCTIONS 1

#assume vec, wrt_vec same sizes

function relative_max_error(vec, wrt_vec, debug=false)
    max_difference = maximum(abs.(vec-wrt_vec))
    max_rel_error = max_difference/maximum(abs.(wrt_vec))
    max_index_diff = argmax(abs.(vec-wrt_vec))
    max_index_wrt_vec = argmax(abs.(wrt_vec))
    if debug
        println("maximum(abs.(wrt_vec): ", maximum(abs.(wrt_vec)))
        println("max_index_diff: ", max_index_diff, ". Total length: ", length(vec), ". x-pos: ", string((max_index_diff/length(vec))) )
        println("max_index_wrt_vec: ", max_index_wrt_vec, ". Total length: ", length(vec), ". x-pos: ", string((max_index_wrt_vec/length(vec))))
    end
    return max_rel_error
end

function relative_l1_error(vec, wrt_vec, debug=false)
    if debug
        println("length(vec): ", length(vec))
        println("length(wrt_vec): ", length(wrt_vec))
    end
    l1 = sum(abs.(vec-wrt_vec)) #*h
    l1_rel_error = l1/sum(abs.(wrt_vec)) # *1/h, which cancels with h from l1
    return l1_rel_error
end

relative_l1_error (generic function with 2 methods)

In [37]:
function restrict_conserve_mass(M_fine)
    # Formula 0 indexing: M^{coarse}_j = (M^{fine}_{2j}*h_{fine} + M^{fine}_{2j-1}*(h_{fine}/2) +M^{fine}_{2j}*(h_{fine}/2))/h_{coarse}
    # Formula 1 indexing: M^{coarse}_j = (M^{fine}_{2j-1}*h_{fine} + M^{fine}_{2j-2}*(h_{fine}/2) +M^{fine}_{2j}*(h_{fine}/2))/h_{coarse}
    #assume restricting to coarser grid with h_coarse = 2*h_fine, i.e. every other node removed.
    #assume M_col even.
    # To conserve mass we need to compute cell averages for M on the coarse grid based on M on the fine grid.    
    println("Must be whole number: ", length(M_fine)/2)
    M_coarse = zeros(Int(length(M_fine)/2))

    M_coarse[1] = M_fine[1]/2 +(M_fine[end]+M_fine[2])/4
    for j in 2:length(M_coarse)
        i=2j-1 # Index transformation from coarse to fine when 1-indxing, which we have here in Julia.
        M_coarse[j] = M_fine[i]/2 +(M_fine[i-1]+M_fine[i+1])/4
    end
    return M_coarse
end


function combine_runs(run_numbers)
    # Dict mapping h → (U_result, M_result)
    results = Dict{Float64, Tuple{Any,Any}}()

    α = x_l = x_r = Δt = t_0 = T = ν = num_it_MFG = num_it_HJB = δ = R = nothing

    required_keys = ["α", "x_l", "x_r", "t_0", "T", "ν", "δ"]
    reference_params = nothing
    for run_number in run_numbers
        params, U_list, M_list = read_run_results(run_number)

        # grab shared parameters (check consistency if you like)
        α, x_l, x_r, Δt, t_0, T, ν, num_it_MFG, num_it_HJB, δ, R =
            (params["α"], params["x_l"], params["x_r"], params["Δt"],
             params["t_0"], params["T"], params["ν"], params["num_it_MFG"],
             params["num_it_HJB"], params["δ"], params["R"])

        h_list = params["h_list"]
        if reference_params === nothing
            reference_params = Dict(k => params[k] for k in required_keys)
        else
            for k in required_keys
                if params[k] != reference_params[k]
                    error("Parameter mismatch for $k between runs: got $(params[k]) but expected $(reference_params[k])")
                end
            end
        end

        for (i, h) in enumerate(h_list)
            if haskey(results, h)
                @warn "Duplicate h detected, overwriting previous results" h=h run_number=run_number
            end
            # Insert or overwrite results for this h
            results[h] = (U_list[i], M_list[i])
        end
    end
    params = (α, x_l, x_r, Δt, t_0, T, ν, δ, R)

    # Sort by h
    sorted_h = sort(collect(keys(results)), rev=true)
    full_h_list = sorted_h
    full_U_list = [results[h][1] for h in sorted_h]
    full_M_list = [results[h][2] for h in sorted_h]
    return full_h_list, full_U_list, full_M_list, params
end

function convergence_rate(full_h_list, full_U_list, full_M_list, params)
    (α, x_l, x_r, Δt, t_0, T, ν, δ, R) = params
    rel_err_U_list = []
    rel_err_M_list = []
    rel_L1_err_M_list = []
    for i in 1:length(full_h_list)-1
        println("")
        println("h: ", full_h_list[i])
        h_i = full_h_list[i]
        h_ip1 = full_h_list[i+1]

        x_coarse = x_l:h_i:(x_r-h_i)
        x_fine   = x_l:h_ip1:(x_r-h_ip1) 
        restriction_step = 2 #Assume we are halving each time. Int(2^(-(log2(h_ip1)-log2(h_i))))

        U_i = full_U_list[i][0 .<= x_coarse .< 1, 1] #x ∈ [0,1), at time t=0.
        U_ip1 = full_U_list[i+1][0 .<= x_fine .< 1, 1]
        U_ip1_restricted = U_ip1[begin:restriction_step:end]
        U_i_rel_err = relative_max_error(U_i, U_ip1_restricted)
        push!(rel_err_U_list, U_i_rel_err)

        M_i = full_M_list[i][0 .<= x_coarse .< 1, end]
        M_ip1 = full_M_list[i+1][0 .<= x_fine .< 1, end]
        println("Whole domain Total mass M_i: ", sum(full_M_list[i][:,end]*full_h_list[i]))
        println("Whole domain Total mass M_ip1: ", sum(full_M_list[i+1][:,end]*full_h_list[i+1]))
        
        M_ip1_restricted = M_ip1[begin:restriction_step:end]
        M_i_rel_err = relative_max_error(M_i, M_ip1_restricted)
        push!(rel_err_M_list, M_i_rel_err)

        M_ip1_L1_restricted = restrict_conserve_mass(M_ip1)
        #Check mass preservation:
        println("Total mass M_i: ", sum(M_i*full_h_list[i]))
        println("Total mass M_ip1: ", sum(M_ip1*full_h_list[i+1]))
        println("Total mass M_ip1_L1_restricted: ", sum(M_ip1_L1_restricted*full_h_list[i]))
        M_i_rel_L1_err = relative_l1_error(M_i, M_ip1_L1_restricted)
        push!(rel_L1_err_M_list, M_i_rel_L1_err)
    end

    conv_rate_U_list = []
    conv_rate_M_list = []
    conv_rate_L1_M_list = []

    for j in 1:length(rel_err_U_list[1:end-1])
        U_rate = log(rel_err_U_list[j]/rel_err_U_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        M_rate = log(rel_err_M_list[j]/rel_err_M_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        M_L1_rate = log(rel_L1_err_M_list[j]/rel_L1_err_M_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        
        push!(conv_rate_U_list, U_rate)
        push!(conv_rate_M_list, M_rate)
        push!(conv_rate_L1_M_list, M_L1_rate)
    end

    #return full_h_list, full_U_list, full_M_list, (α, x_l, x_r, t_0, T, ν, δ)
    return rel_err_U_list, rel_err_M_list, rel_L1_err_M_list, conv_rate_U_list, conv_rate_M_list, conv_rate_L1_M_list
end


convergence_rate (generic function with 1 method)

In [1]:
function ref_sol_restrict_conserve_mass(M_fine, restriction_step)
    # Formula 0 indexing: M^{coarse}_j = (M^{fine}_{2j}*h_{fine} + M^{fine}_{2j-1}*(h_{fine}/2) +M^{fine}_{2j}*(h_{fine}/2))/h_{coarse}
    # Formula 1 indexing: M^{coarse}_j = (M^{fine}_{2j-1}*h_{fine} + M^{fine}_{2j-2}*(h_{fine}/2) +M^{fine}_{2j}*(h_{fine}/2))/h_{coarse}
    #assume restricting to coarser grid with h_coarse = 2*h_fine, i.e. every other node removed.
    #assume M_col even.
    # To conserve mass we need to compute cell averages for M on the coarse grid based on M on the fine grid.    
    println("log2(restriction_step): ", log2(restriction_step))
    halvings = Int(log2(restriction_step))
    println("Halvings: ", halvings)

    #println("Must be whole number: ", length(M_fine)/2)
    M = M_fine
    for i in 1:halvings
        M_coarse = zeros(Int(length(M)/2))

        M_coarse[1] = M[1]/2 +(M[end]+M[2])/4
        for j in 2:length(M_coarse)
            i=2j-1 # Index transformation from coarse to fine when 1-indxing, which we have here in Julia.
            M_coarse[j] = M[i]/2 +(M[i-1]+M[i+1])/4
        end
        M=M_coarse
    end
    return M
end

function ref_sol_convergence_rate(full_h_list, full_U_list, full_M_list, params)
    (α, x_l, x_r, Δt, t_0, T, ν, δ, R) = params
    rel_err_U_list = []
    rel_err_M_list = []
    rel_L1_err_M_list = []
    for i in 1:length(full_h_list)-1
        println("")
        println("h: ", full_h_list[i])
        h_i = full_h_list[i]
        h_finest = full_h_list[end]

        x_coarse = x_l:h_i:(x_r-h_i)
        x_fine   = x_l:h_finest:(x_r-h_finest) 
        restriction_step = Int(2^(-(log2(h_finest)-log2(h_i)))) #Assume we are halving each time. Int(2^(-(log2(h_finest)-log2(h_i))))
        println("i: ", i, ". Restriction_step: ", restriction_step)

        U_i = full_U_list[i][0 .<= x_coarse .< 1, 1] #x ∈ [0,1), at time t=0.
        U_finest = full_U_list[end][0 .<= x_fine .< 1, 1]
        U_finest_restricted = U_finest[begin:restriction_step:end]
        U_i_rel_err = relative_max_error(U_i, U_finest_restricted)
        push!(rel_err_U_list, U_i_rel_err)

        M_i = full_M_list[i][0 .<= x_coarse .< 1, end]
        M_finest = full_M_list[end][0 .<= x_fine .< 1, end]
        println("Whole domain Total mass M_i: ", sum(full_M_list[i][:,end]*full_h_list[i]))
        println("Whole domain Total mass M_finest: ", sum(full_M_list[end][:,end]*full_h_list[end]))
        
        M_finest_restricted = M_finest[begin:restriction_step:end]
        M_i_rel_err = relative_max_error(M_i, M_finest_restricted)
        push!(rel_err_M_list, M_i_rel_err)

        M_finest_L1_restricted = ref_sol_restrict_conserve_mass(M_finest, restriction_step)
        #Check mass preservation:
        println("Total mass M_i: ", sum(M_i*full_h_list[i]))
        println("Total mass M_finest: ", sum(M_finest*full_h_list[end]))
        println("Total mass M_finest_L1_restricted: ", sum(M_finest_L1_restricted*full_h_list[i]))
        M_i_rel_L1_err = relative_l1_error(M_i, M_finest_L1_restricted)
        push!(rel_L1_err_M_list, M_i_rel_L1_err)
    end

    conv_rate_U_list = []
    conv_rate_M_list = []
    conv_rate_L1_M_list = []

    for j in 1:length(rel_err_U_list[1:end-1])
        U_rate = log(rel_err_U_list[j]/rel_err_U_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        M_rate = log(rel_err_M_list[j]/rel_err_M_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        M_L1_rate = log(rel_L1_err_M_list[j]/rel_L1_err_M_list[j+1])/(log(full_h_list[j]/full_h_list[j+1]))
        
        push!(conv_rate_U_list, U_rate)
        push!(conv_rate_M_list, M_rate)
        push!(conv_rate_L1_M_list, M_L1_rate)
    end

    #return full_h_list, full_U_list, full_M_list, (α, x_l, x_r, t_0, T, ν, δ)
    return rel_err_U_list, rel_err_M_list, rel_L1_err_M_list, conv_rate_U_list, conv_rate_M_list, conv_rate_L1_M_list
end

ref_sol_convergence_rate (generic function with 1 method)

In [ ]:
#PRINT U AND M CONVERGENCE FOR LATEX

### 
SPECIFY_RUNS = [...] # e.g. [2,12,18] for runs 2, 12, 18
full_h_list, full_U_list, full_M_list, params = combine_runs(SPECIFY_RUNS)
for i in 1:length(full_h_list)
    println("i: ", i)
    println("h: ", full_h_list[i])
    println("size(U): ", size(full_U_list[i]))
    println("size(M): ", size(full_M_list[i]))
end
println("params: ", params)

ref_sol_rel_err_U_list, ref_sol_rel_err_M_list, ref_sol_rel_L1_err_M_list, ref_sol_conv_rate_U_list, ref_sol_conv_rate_M_list, ref_sol_conv_rate_L1_M_list = ref_sol_convergence_rate(full_h_list, full_U_list, full_M_list, params)

###

using Printf
# Print LaTeX table header
println("\\begin{tabular}{|c|c|c|c|c|c|c|}")
println("\\hline")
println("\$h\$ & U rel. \$L^{\\infty}\$ error  & U rate & M rel. \$L^{1}\$ error & M rate (\$L^1\$) & M rel. \$L^{\\infty}\$ error & M rate (\$L^{\\infty}\$) \\\\")
println("\\hline")

# Print the rows of the table
for i in 1:length(full_h_list)-1
    U_re_s_num = @sprintf "%.2e" ref_sol_rel_err_U_list[i]    
    M_re_s_num = @sprintf "%.2e" ref_sol_rel_L1_err_M_list[i]
    M_max_re_s_num = @sprintf "%.2e" ref_sol_rel_err_M_list[i]

    if i==1
        U_ra_s_num = "--"
        M_ra_s_num = "--"
        M_max_ra_s_num = "--"
        println("\$2^{",string(Int(log2(full_h_list[i])),"}\$", " & ", "\$",U_re_s_num,"\$"," & ", "--", " & ", "\$",M_re_s_num,"\$", " & ","--"," & ","\$", M_max_re_s_num,"\$", " & ","--", " \\\\"))
        println("\\hline")
    else
        U_ra_s_num = @sprintf "%.2f" ref_sol_conv_rate_U_list[i-1]
        M_ra_s_num = @sprintf "%.2f" ref_sol_conv_rate_L1_M_list[i-1]
        M_max_ra_s_num = @sprintf "%.2f" ref_sol_conv_rate_M_list[i-1]
    
        println("\$2^{",string(Int(log2(full_h_list[i])),"}\$", " & ", "\$",U_re_s_num,"\$"," & ", "\$",U_ra_s_num,"\$", " & ", "\$",M_re_s_num,"\$", " & ","\$", M_ra_s_num,"\$"," & ","\$", M_max_re_s_num,"\$", " & ","\$", M_max_ra_s_num,"\$", " \\\\"))
        println("\\hline")
    end
end

# Print LaTeX table footer
println("\\end{tabular}")

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
$h$ & U rel. $L^{\infty}$ error  & U rate & M rel. $L^{1}$ error & M rate ($L^1$) & M rel. $L^{\infty}$ error & M rate ($L^{\infty}$) \\
\hline
$2^{-5}$ & $4.38e-03$ & -- & $1.34e-01$ & -- & $1.38e-01$ & -- \\
\hline
$2^{-6}$ & $4.88e-04$ & $3.17$ & $3.31e-02$ & $2.02$ & $3.69e-02$ & $1.91$ \\
\hline
$2^{-7}$ & $1.05e-05$ & $5.53$ & $8.17e-03$ & $2.02$ & $8.87e-03$ & $2.06$ \\
\hline
$2^{-8}$ & $2.65e-06$ & $1.99$ & $1.96e-03$ & $2.06$ & $2.13e-03$ & $2.06$ \\
\hline
$2^{-9}$ & $6.23e-07$ & $2.09$ & $3.95e-04$ & $2.31$ & $4.27e-04$ & $2.32$ \\
\hline
\end{tabular}
